# Structure Visualization

This notebook demonstrates COMPASS visualization capabilities:

1. Build pixel stacks with different configurations
2. 2D cross-section plots (XZ and XY planes)
3. Compare different pixel pitches and Bayer patterns
4. Explore microlens parameter effects
5. Interactive 3D viewer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from compass.core.config_schema import load_config
from compass.geometry.pixel_stack import PixelStack
from compass.geometry.builder import GeometryBuilder
from compass.materials.database import MaterialDB
from compass.visualization.structure_plot_2d import plot_structure_xz, plot_structure_xy
from compass.visualization.viewer_3d import StructureViewer3D

In [ ]:
mat_db = MaterialDB()

## 1. Build Multiple Pixel Configurations

We load several configs to compare different pixel designs:
- 1.0 um pitch BSI pixel
- 0.7 um pitch BSI pixel
- 1.4 um pitch BSI pixel with Bayer CFA

In [ ]:
configs = {
    "1.0um BSI": load_config(Path("../configs/pixel/default_bsi_1um.yaml")),
    "0.7um BSI": load_config(Path("../configs/pixel/bsi_0p7um.yaml")),
    "1.4um Bayer": load_config(Path("../configs/pixel/bsi_1p4um_bayer.yaml")),
}

stacks = {}
for name, cfg in configs.items():
    builder = GeometryBuilder(cfg.pixel, mat_db)
    stacks[name] = builder.build()
    print(f"{name}: pitch={cfg.pixel.pitch_x}x{cfg.pixel.pitch_y} um, "
          f"layers={len(stacks[name].layer_slices)}, "
          f"thickness={stacks[name].total_thickness:.2f} um")

## 2. XZ Cross-Section Plots

The XZ plane shows the vertical structure of the pixel stack.
Light propagates in the **-z** direction (from top to bottom in BSI).

Coordinate convention:
- x, y: lateral (in-plane)
- z: stack direction, air at z_max, silicon at z_min

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, (name, stack) in zip(axes, stacks.items()):
    cfg = configs[name]
    plot_structure_xz(
        stack,
        y_cut=cfg.pixel.pitch_y / 2,
        wavelength=0.550,
        mat_db=mat_db,
        show_materials=True,
        ax=ax,
    )
    ax.set_title(name)

fig.suptitle("Pixel Structure Comparison (XZ cross-section at y = pitch/2)", fontsize=14)
fig.tight_layout()

## 3. XY Cross-Section Plots

The XY plane shows the lateral structure at a given z-height.
Useful for inspecting color filter arrays, metal grids, and DTI patterns.

In [ ]:
# Show XY slices at different z-heights for the Bayer pixel
bayer_stack = stacks["1.4um Bayer"]
bayer_cfg = configs["1.4um Bayer"]

# Choose z-heights: microlens, color filter, photodiode
z_positions = {
    "Microlens (top)": bayer_stack.z_max - 0.1,
    "Color Filter": bayer_stack.z_max - 1.5,
    "Photodiode (Si)": bayer_stack.z_min + 0.5,
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (label, z_val) in zip(axes, z_positions.items()):
    plot_structure_xy(
        bayer_stack,
        z_cut=z_val,
        wavelength=0.550,
        mat_db=mat_db,
        show_materials=True,
        ax=ax,
    )
    ax.set_title(f"{label} (z = {z_val:.2f} um)")

fig.suptitle("1.4um Bayer Pixel - XY Slices at Different Heights", fontsize=14)
fig.tight_layout()

## 4. Bayer Pattern Visualization

For a 2x2 Bayer unit cell, the color filter assignment follows the RGGB pattern.
Each pixel gets a different filter material.

In [ ]:
# Visualize the Bayer color filter pattern
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Real part of permittivity at CFA layer for different wavelengths
for ax, wl, wl_name in zip(axes, [0.450, 0.650], ["Blue (450nm)", "Red (650nm)"]):
    plot_structure_xy(
        bayer_stack,
        z_cut=bayer_stack.z_max - 1.5,  # CFA layer height
        wavelength=wl,
        mat_db=mat_db,
        show_materials=True,
        ax=ax,
    )
    ax.set_title(f"Bayer CFA at {wl_name}")

fig.tight_layout()

## 5. Microlens Parameter Effects

The microlens uses a superellipse profile:

$$z(x, y) = h \cdot (1 - r^2)^{1/2\alpha}$$

where $r$ is the normalized radial coordinate, $h$ is the lens height,
and $\alpha$ controls the profile shape (1 = spherical, >1 = flatter top).

In [ ]:
from compass.core.config_schema import load_config, override_config

# Vary the superellipse exponent alpha
alpha_values = [0.5, 1.0, 2.0]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

base_cfg = configs["1.0um BSI"]

for ax, alpha in zip(axes, alpha_values):
    # Override the microlens alpha parameter
    cfg = override_config(base_cfg, {"pixel.microlens.alpha": alpha})
    builder = GeometryBuilder(cfg.pixel, mat_db)
    stack = builder.build()

    plot_structure_xz(
        stack,
        y_cut=cfg.pixel.pitch_y / 2,
        wavelength=0.550,
        mat_db=mat_db,
        show_materials=True,
        ax=ax,
    )
    ax.set_title(f"Microlens alpha = {alpha}")

fig.suptitle("Superellipse Microlens Profile Comparison", fontsize=14)
fig.tight_layout()

In [ ]:
# Plot the 1D microlens profile for different alpha values
r = np.linspace(0, 0.95, 200)
h = 0.5  # typical microlens height in um

fig, ax = plt.subplots(figsize=(8, 4))
for alpha in [0.5, 1.0, 1.5, 2.0, 3.0]:
    z_profile = h * (1 - r**2) ** (1 / (2 * alpha))
    ax.plot(r, z_profile, label=f"alpha = {alpha}")

ax.set_xlabel("Normalized radial coordinate r")
ax.set_ylabel("Height z (um)")
ax.set_title("Superellipse Microlens Profiles")
ax.legend()
ax.grid(True, alpha=0.3)

## 6. 3D Structure Viewer

COMPASS includes an interactive 3D viewer for inspecting the full pixel stack.
This uses a voxelized representation of the permittivity distribution.

In [ ]:
# Launch 3D viewer for the Bayer pixel
viewer = StructureViewer3D(bayer_stack, mat_db=mat_db, wavelength=0.550)

# Render with default settings (shows all layers, materials color-coded)
viewer.show(
    opacity=0.6,
    show_grid=True,
    show_layer_boundaries=True,
)

## Summary

Visualization tools demonstrated:

| Function | Description |
|---|---|
| `plot_structure_xz()` | Vertical cross-section (XZ plane) |
| `plot_structure_xy()` | Horizontal cross-section (XY plane) |
| `StructureViewer3D` | Interactive 3D voxel viewer |

Key pixel design parameters explored:
- Pixel pitch (0.7, 1.0, 1.4 um)
- Bayer CFA pattern (RGGB)
- Microlens superellipse exponent alpha

Next: See `03_solver_comparison.ipynb` for multi-solver QE comparison.